In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

data=tf.keras.datasets.mnist.load_data()   #download MNIST dataset
(train_images,train_labels),(test_images,test_labels)=data    #convert all the data into 4 tuples

train_images=np.expand_dims(train_images,axis=-1)
train_images=train_images.astype('float32')
train_images=((train_images/255.0)*2)-1  #normalize the train images to the range [-1, 1]

test_images=np.expand_dims(test_images,axis=-1)
test_images=test_images.astype('float32')
test_images=((test_images/255.0)*2)-1  #normalize the test images to the range [-1, 1]

G=tf.keras.Sequential([
    tf.keras.layers.Dense(units=12544,input_shape=(100,)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Reshape((7,7,256)),
    tf.keras.layers.Conv2DTranspose(filters=128,kernel_size=(3,3),padding="same",strides=(2,2)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Conv2DTranspose(filters=1,kernel_size=(3,3),activation="tanh",padding="same",strides=(2,2))
])

G.summary()

D=tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),padding="same",strides=(2,2),input_shape=(28,28,1)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Conv2D(filters=128,kernel_size=(3,3),padding="same",strides=(2,2)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=256,activation="relu"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=1)
])

D.summary()



In [ ]:
from scipy.linalg import sqrtm

GP_WEIGHT=10.0 # Standard value for the Gradient Penalty weight
bestfid=float('inf')
save_path='best_generator.weights.h5' 
batch_size=256
epochs=100
latent_dim=100 
D_loss_mean=[]
G_loss_mean=[]
D_optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.5)
G_optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.5)
dataset=tf.data.Dataset.from_tensor_slices(train_images).shuffle(len(train_images)).batch(batch_size)

@tf.function
def Train_Step_WGANGP(real_images):
    batch_size = tf.shape(real_images)[0]

    noise = tf.random.normal(shape=(batch_size, latent_dim))

    with tf.GradientTape() as disc_tape:
        fake_images = G(noise, training=True)

        real_output = D(real_images, training=True)
        fake_output = D(fake_images, training=True)

        # Loss WGAN
        disc_wgan_loss = tf.reduce_mean(fake_output) - tf.reduce_mean(real_output)

        # Gradient Penalty
        alpha = tf.random.uniform(shape=[batch_size, 1, 1, 1], minval=0., maxval=1.)
        interpolated_images = alpha * real_images + (1 - alpha) * fake_images

        with tf.GradientTape() as gp_tape:
            gp_tape.watch(interpolated_images)
            pred = D(interpolated_images, training=True)

        grads = gp_tape.gradient(pred, [interpolated_images])[0]
        norm = tf.sqrt(tf.reduce_sum(tf.square(grads), axis=[1, 2, 3]))
        gradient_penalty = tf.reduce_mean((norm - 1.0) ** 2)

        disc_loss = disc_wgan_loss + GP_WEIGHT * gradient_penalty

    D_Gradient = disc_tape.gradient(disc_loss, D.trainable_variables)
    D_optimizer.apply_gradients(zip(D_Gradient, D.trainable_variables))

    # traning G
    with tf.GradientTape() as gen_tape:
        noise = tf.random.normal(shape=(batch_size, latent_dim))
        fake_images = G(noise, training=True)
        fake_output = D(fake_images, training=True)

        # Loss del Generatore
        gen_loss = -tf.reduce_mean(fake_output)

    G_Gradient = gen_tape.gradient(gen_loss, G.trainable_variables)
    G_optimizer.apply_gradients(zip(G_Gradient, G.trainable_variables))

    return disc_loss, gen_loss
for epoch in range(epochs):
    epoch_d_loss = []
    epoch_g_loss = []

    for image_batch in dataset:
        d_loss, g_loss = Train_Step_WGANGP(image_batch)
        epoch_d_loss.append(d_loss.numpy())
        epoch_g_loss.append(g_loss.numpy())

    avg_d = np.mean(epoch_d_loss)
    avg_g = np.mean(epoch_g_loss)

    D_loss_mean.append(avg_d)
    G_loss_mean.append(avg_g)

    print(f"Epoch {epoch + 1}/{epochs} - D Loss: {avg_d:.4f}, G Loss: {avg_g:.4f}")




In [ ]:
#plot of G and L mean loss for all 30 epochs

plt.figure(figsize=(10,6))
plt.plot(range(1,epochs+1),D_loss_mean,"o-",label="D Loss",color="red")
plt.plot(range(1,epochs+1),G_loss_mean,"o-",label="G Loss",color="blue")
plt.legend()
plt.xticks(ticks=range(1,epochs+1))
plt.title("Loss Comparation")
plt.grid(True)
plt.xlabel("epochs")
plt.ylabel("loss")
plt.show()

In [ ]:
print("D_loss value:")
for el in D_loss_mean:
    print(el)
print("G_loss value:")
for el in G_loss_mean:
    print(el)

In [ ]:
latent_dim = 150

def generate_and_display_images(model, num_images_to_generate):
    noise = tf.random.normal([num_images_to_generate, latent_dim])
    generated_images = model(noise, training=False)
    generated_images = (generated_images + 1) / 2.0

    grid_size = int(np.sqrt(num_images_to_generate))
    fig, axes = plt.subplots(grid_size, grid_size, figsize=(8, 8))

    axes = axes.flatten()

    for i in range(num_images_to_generate):
        img = generated_images[i, :, :, 0]

        axes[i].imshow(img, cmap='gray')
        axes[i].axis('off')

    plt.tight_layout()
    plt.show()

generate_and_display_images(G, 150)

In [ ]:
def calculate_fid(model, real_images, fake_images):     #function for calculate the FID value
    
    act1 = model.predict(real_images)
    act2 = model.predict(fake_images)

    mu1, sigma1 = act1.mean(axis=0), np.cov(act1, rowvar=False)
    mu2, sigma2 = act2.mean(axis=0), np.cov(act2, rowvar=False)

    ssdiff = np.sum((mu1 - mu2)**2.0)
    covmean = sqrtm(sigma1.dot(sigma2))

    if np.iscomplexobj(covmean):
        covmean = covmean.real

    fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid

#image preparation for FID evaluation
NUM_IMAGES_FOR_FID = 10000 

real_images_subset = train_images[:NUM_IMAGES_FOR_FID]
real_images_for_fid = (real_images_subset * 127.5 + 127.5).astype('uint8')
real_images_resized = tf.image.resize(real_images_for_fid, [75, 75]).numpy()
real_images_rgb = np.repeat(real_images_resized, 3, axis=-1)
real_images_preprocessed = tf.keras.applications.inception_v3.preprocess_input(real_images_rgb)

noise = tf.random.normal([NUM_IMAGES_FOR_FID, latent_dim])
fake_images = G(noise, training=False)
fake_images_for_fid = (fake_images * 127.5 + 127.5).numpy().astype('uint8')
fake_images_resized = tf.image.resize(fake_images_for_fid, [75, 75]).numpy()
fake_images_rgb = np.repeat(fake_images_resized, 3, axis=-1)
fake_images_preprocessed = tf.keras.applications.inception_v3.preprocess_input(fake_images_rgb)


inception_model = tf.keras.applications.InceptionV3(
    include_top=False,
    pooling='avg',
    input_shape=(75, 75, 3)
)

fid_score = calculate_fid(inception_model, real_images_preprocessed, fake_images_preprocessed)

print("\n" + "="*30)
print(f"Final FID Score: {fid_score:.2f}")
print("="*30)

## S&P500